In [ ]:
pip install yfinance

In [ ]:
pip install yahoofinancials

In [ ]:
pip install plotly.express==0.4.0

In [ ]:
pip install stldecompose

In [ ]:
pip install statsmodels==0.10.2

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import yfinance as yf
from yahoofinancials import YahooFinancials
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
#import plotly.express as px
from stldecompose import decompose
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

VERİ OKUMA İŞLEMİ

In [ ]:
yf=yf.download("AAPL",
              start="2017-01-01",
              progress=False)
yf.index=pd.to_datetime(yf.index)

In [ ]:
fig= go.Figure(data=[go.Candlestick(x=yf.index,
                                    open=yf["Open"],
                                    high=yf["High"],
                                    low=yf["Low"],
                                    close=yf["Close"])])
fig.show()

In [ ]:
#fig = px.line(yf, x =yf.Date,y=yf.index, title='Apple Share Prices over time (2014)')
#fig.show()

In [ ]:
yf.describe()

![](http://)****Time Series Decomposition

Total Manner
Yt=Tt+St+Rt

In [ ]:
yf_close = yf["Close"]
decomp = decompose(yf_close.values)
decomp.plot().show()


In [ ]:
decomp.seasonal

In [ ]:
decomp.seasonal[decomp.seasonal<-4.66]


In [ ]:
decomp = decompose(yf_close.values)
decomp.plot().show()

positions=np.arange(1,len(yf_close),90)
labels=positions

plt.xticks(positions,labels)


In [ ]:
trend=decomp.trend
seasonal=decomp.seasonal
residual=decomp.resid

In [ ]:
seasonal

# MOVING AVERAGE METHOD

In [ ]:
#(len(yf)//2)
yf_close=yf.iloc[:(len(yf)//2),3]
yf_close

In [ ]:
rolling_app5 = yf_close.rolling(window=5).mean()
rolling_app14 = yf_close.rolling(window=14).mean()
rolling_app21 = yf_close.rolling(window=21).mean()

In [ ]:
MAs = pd.concat([yf_close,rolling_app5, rolling_app14, rolling_app21],axis=1)
MAs.columns=["Close","short","mid","long"]
MAs.head(30)


In [ ]:
MAs.head()

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))
ax.plot(yf_close.index,yf_close,label="Apple")
ax.plot(rolling_app5.index,rolling_app5,label="5 days rolling")
ax.plot(rolling_app14.index,rolling_app14,label="14 days rolling")
ax.plot(rolling_app21.index,rolling_app21,label="21 days rolling")
ax.legend(loc="upper left")

In [ ]:
MAs.dropna(axis=0,inplace=True)
MAs.head(10)

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))
ax.plot(yf_close.index,yf_close,label="Apple")
ax.plot(rolling_app5.index,rolling_app5,label="5 days rolling")
ax.plot(rolling_app14.index,rolling_app14,label="14 days rolling")
ax.plot(rolling_app21.index,rolling_app21,label="21 days rolling")
ax.legend(loc="upper left")

In [ ]:
np.abs(np.percentile(np.array(MAs["short"]- MAs["mid"]),10))#veri setini sıralayıp %10 unu aldırıyoruz.#np.abs = mutlak değerini al


BOLLINGER BAND

In [ ]:
def buy_sell_MA(data,opt=50):
    buy_sell= []
    buy_signal= []
    sell_signal= []
    flag = 42
    
    up= data[(np.array(data["short"]- data["mid"])>0)& (np.array(data["short"]-data["long"])>0)]
    down = data[(np.array(data["short"]- data["mid"])<0)& (np.array(data["short"]-data["long"])<0)]
    
    ups=np.percentile(np.array(up["short"]-up["mid"]),opt)
    downs=np.percentile(np.array(down["short"]-down["long"]),opt)
    
    for i in range(0,len(data)):
        if (data["short"][i]>data["mid"][i]+ups) & (data["short"][i]>data["long"][i]+ups):
            buy_signal.append(np.nan)
            if flag != 1: #flag e 42 demiştik. ! ile soruyorum 42 mi değil
                sell_signal.append(data["Close"][i])#append liste sonuna eleman ekler
                buy_sell.append(data["Close"][i])
                flag=1
            else:
                sell_signal.append(np.nan)
        elif (data["short"][i]<data["mid"][i]-downs) & (data["short"][i]<data["long"][i]-downs):
            sell_signal.append(np.nan)
            if flag !=0:
                buy_signal.append(data["Close"][i])
                buy_sell.append(-data["Close"][i])
                flag=0
            else:
                buy_signal.append(np.nan)
        else:
            buy_sell.append(np.nan)
            sell_signal.append(np.nan)
            buy_signal.append(np.nan)
            
    operations = np.array(buy_sell)
    operations = operations[~np.isnan(operations)]
    
    neg=0
    pos=0
    
    for i in range(len(operations)):
        if operations[i]<0:
            neg=i
            break
    for i in range(1,len(operations)):
        if operations[-i]>0:
            pos=i-1
            break
    operations=operations[neg:-pos]
    PL=np.sum(operations)
    return(buy_signal,sell_signal,PL)

In [ ]:
MAs["BUY"]=buy_sell_MA(MAs)[0]
MAs["SELL"]=buy_sell_MA(MAs)[1]

In [ ]:
buy_sell_MA(MAs)[2]

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(MAs.index, MAs["BUY"], color="green",label="BUY", marker="^",alpha=1)
plt.scatter(MAs.index, MAs["SELL"], color="red",label="SELL", marker="v",alpha=1)
plt.plot(MAs["Close"],label="Close Price",alpha=0.5)
plt.title("Close Price Buy and Sell Signals")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend(loc="upper left")
plt.show()

opt=50 olan bölümü i ye çevirdim(optimize ettim)

In [ ]:
np.arange(10,91,5)

In [ ]:
opt = []
percentile=np.arange(10,91,5)
for i,j in enumerate(percentile):
    opt.append((percentile[i],buy_sell_MA(MAs,j)[2]))
opt=pd.DataFrame(opt)
opt.columns=["Percentile","Performance"]
opt=opt.sort_values(by="Performance",ascending=False)

In [ ]:
MAs["BUY"]=buy_sell_MA(MAs,opt["Percentile"].values[0])[0]
MAs["SELL"]=buy_sell_MA(MAs,opt["Percentile"].values[0])[1]

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(MAs.index, MAs["BUY"], color="green",label="BUY", marker="^",alpha=1)
plt.scatter(MAs.index, MAs["SELL"], color="red",label="SELL", marker="v",alpha=1)
plt.plot(MAs["Close"],label="Close Price",alpha=0.5)
plt.title("Close Price Buy and Sell Signals")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend(loc="upper left")
plt.show()